In [1]:
from sklearn.neighbors import KDTree

In [2]:
import pandas as pd

In [13]:
# read in datasets
df1 = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Spotify-Song-Suggester/data/SpotifyAudioFeaturesApril2019.csv')
df2 = pd.read_csv('/Users/ianforrest/Desktop/coding/repos/DS_Spotify-Song-Suggester/data/SpotifyAudioFeaturesNov2018.csv')

# check shapes
df1.shape, df2.shape

((130663, 17), (116372, 17))

In [14]:
# merge dataframes together
df = pd.concat([df1,df2])
df.shape

(247035, 17)

In [15]:
df.head()

artist_name                track_id  \
0          YG  2RM4jf1Xa9zPgMGRDiht8O   
1          YG  1tHDG53xJNGsItRA3vfVgs   
2       R3HAB  6Wosx2euFPMT14UXiWudMy   
3  Chris Cooq  3J2Jpw61sO7l6Hc7qdYV91   
4  Chris Cooq  2jbYvQCyPgX3CdmAzeVeuS   

                                       track_name  acousticness  danceability  \
0  Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj      0.005820         0.743   
1                    BAND DRUM (feat. A$AP Rocky)      0.024400         0.846   
2                                   Radio Silence      0.025000         0.603   
3                                         Lactose      0.029400         0.800   
4                             Same - Original mix      0.000035         0.783   

   duration_ms  energy  instrumentalness  key  liveness  loudness  mode  \
0       238373   0.339             0.000    1    0.0812    -7.678     1   
1       214800   0.557             0.000    8    0.2860    -7.259     1   
2       138913   0.723             0.000    9    0.0824    -5.890     0   
3       125381   0.579             0.912    5    0.0994   -12.118     0   
4       124016   0.792             0.878    7    0.0332   -10.277     1   

   speechiness    tempo  time_signature  valence  popularity  
0       0.4090  203.927               4    0.118          15  
1       0.4570  159.009               4    0.371           0  
2       0.0454  114.966               4    0.382          56  
3       0.0701  123.003               4    0.641           0  
4       0.0661  120.047               4    0.928           0

In [16]:
# Note that tracks apear multiple times in the dataset

In [17]:
df.iloc[2]

artist_name                          R3HAB
track_id            6Wosx2euFPMT14UXiWudMy
track_name                   Radio Silence
acousticness                         0.025
danceability                         0.603
duration_ms                         138913
energy                               0.723
instrumentalness                         0
key                                      9
liveness                            0.0824
loudness                             -5.89
mode                                     0
speechiness                         0.0454
tempo                              114.966
time_signature                           4
valence                              0.382
popularity                              56
Name: 2, dtype: object

In [18]:
df.iloc[121994]

artist_name                                      R3HAB
track_id                        6Wosx2euFPMT14UXiWudMy
track_name          Radio Silence (With Jocelyn Alice)
acousticness                                     0.025
danceability                                     0.603
duration_ms                                     138913
energy                                           0.723
instrumentalness                                     0
key                                                  9
liveness                                        0.0824
loudness                                         -5.89
mode                                                 0
speechiness                                     0.0454
tempo                                          114.966
time_signature                                       4
valence                                          0.382
popularity                                          56
Name: 121994, dtype: object

In [19]:
len(df)

247035

In [20]:
len(df.drop_duplicates(subset = 'track_id'))

130989

In [21]:
df = df.drop_duplicates(subset = 'track_id').reset_index().drop(['index'], axis=1)

In [22]:
# Features to be trained on
df_feats = df.drop(['artist_name', 'track_id', 'track_name', 'popularity'], axis=1)
# Identifying information, Artist, ID, Name
df_atts = df[['artist_name', 'track_id', 'track_name']]

In [23]:
# Dictionary mapping track ID to dataframe index
val_ind_map = {value : index for index, value in zip(range(len(df_atts)), df_atts['track_id'])}

In [24]:
val_ind_map

{'2RM4jf1Xa9zPgMGRDiht8O': 0,
 '1tHDG53xJNGsItRA3vfVgs': 1,
 '6Wosx2euFPMT14UXiWudMy': 2,
 '3J2Jpw61sO7l6Hc7qdYV91': 3,
 '2jbYvQCyPgX3CdmAzeVeuS': 4,
 '26Y1lX7ZOpw9Ql3gGAlqLK': 5,
 '5eIyK73BrxHLnly4F9PWqg': 6,
 '13Mf2ZBpfNkgWJowvM5hXh': 7,
 '7BQaRTHk44DkMhIVNcXy2D': 8,
 '049RxG2laEl9U1PGYeIqLV': 9,
 '118GQ70Sp6pMqn6w1oKuki': 10,
 '6S7cr72a7a8RVAXzDCRj6m': 11,
 '7h2qWpMJzIVtiP30E8VDW4': 12,
 '3KVQFxJ5CWOcbxdpPYdi4o': 13,
 '0JjNrI1xmsTfhaiU1R6OVc': 14,
 '3HjTcZt29JUHg5m60QhlMw': 15,
 '42LWRdkWxM9aWmDImWvH6C': 16,
 '32dMH9MvlTJaABrPHY52Yb': 17,
 '5RCPsfzmEpTXMCTNk7wEfQ': 18,
 '0y0mwXrdEzjUK5Nq8GDPnY': 19,
 '3RSMqu36JZnmMkrnNmnqyd': 20,
 '1o0fkWCltFHVeFIRHqvR5b': 21,
 '2iGShSeV6WcDbez5SLJ2bJ': 22,
 '2rNTo0tGUMW6rn0uHzV5er': 23,
 '5Egkx8edirN0pR2R58C2ME': 24,
 '67r3lnzstENsRYlZWq6DYP': 25,
 '4X8W9SSu9D5MujoxwIwqw6': 26,
 '4lncSzeN8WOH2iHEO593iZ': 27,
 '1L67mcddFQ65MfA3wO3MHV': 28,
 '21DU83QG4jB4mQKh76X32h': 29,
 '08nyEVO684j7pcTAhEY2zJ': 30,
 '4LMVmlX8WXPu8OyPwzkNpR': 31,
 '7JYCpIzpoidDOnnm

In [25]:
# Train tree on data
tree = KDTree(df_feats, leaf_size=10)

In [26]:
# Tree can be queried for nearest indices (notice the nearest neighbor is the input itself)
tree.query(pd.DataFrame(df_feats.iloc[11]).T, k=11, return_distance=False)

array([[   11,    23,  2293,  2069, 37270, 70631,  6281, 75678, 75774,
        11122, 26890]])

In [27]:
# We can define a function to query based on ID
def nearest_by_id(id, k=3):
    return tree.query(df_feats.iloc[[val_ind_map[id]]], k=k)[1]

In [28]:
val_ind_map['6Wosx2euFPMT14UXiWudMy']

2

In [29]:
nearest_by_id('6Wosx2euFPMT14UXiWudMy')

array([[    2, 38628,  6498]])

In [30]:
df_feats.iloc[2]

acousticness             0.0250
danceability             0.6030
duration_ms         138913.0000
energy                   0.7230
instrumentalness         0.0000
key                      9.0000
liveness                 0.0824
loudness                -5.8900
mode                     0.0000
speechiness              0.0454
tempo                  114.9660
time_signature           4.0000
valence                  0.3820
Name: 2, dtype: float64

In [31]:
df_feats.iloc[441]

acousticness             0.302
danceability             0.496
duration_ms         138031.000
energy                   0.550
instrumentalness         0.811
key                      6.000
liveness                 0.254
loudness                -3.404
mode                     1.000
speechiness              0.251
tempo                  174.237
time_signature           4.000
valence                  0.851
Name: 441, dtype: float64

In [32]:
df_feats.iloc[1004]

acousticness             0.40200
danceability             0.84200
duration_ms         139090.00000
energy                   0.77500
instrumentalness         0.00021
key                      1.00000
liveness                 0.14500
loudness                -5.74800
mode                     1.00000
speechiness              0.10100
tempo                  115.52000
time_signature           4.00000
valence                  0.93600
Name: 1004, dtype: float64

In [33]:
# Note that many of the attributes aren't close at all.
# Large numbers, as in the durration and tempo, dominate the classification.

In [34]:
from sklearn.preprocessing import StandardScaler

In [35]:
spoty_scalar = StandardScaler().fit(df_feats)
# Note that the track ids are being used as an index, meaning we don't need val_ind_map anymore
df_stand = pd.DataFrame(spoty_scalar.transform(df_feats),
                        columns=df_feats.columns,
                        index=df_atts['track_id'].values)

In [36]:
df_feats

acousticness  danceability  duration_ms  energy  instrumentalness  \
0           0.005820         0.743       238373   0.339             0.000   
1           0.024400         0.846       214800   0.557             0.000   
2           0.025000         0.603       138913   0.723             0.000   
3           0.029400         0.800       125381   0.579             0.912   
4           0.000035         0.783       124016   0.792             0.878   
...              ...           ...          ...     ...               ...   
130984      0.820000         0.418       235771   0.204             0.936   
130985      0.070400         0.614       209032   0.952             0.000   
130986      0.064000         0.343       324920   0.487             0.000   
130987      0.029200         0.380       423377   0.399             0.000   
130988      0.101000         0.834       240353   0.517             0.000   

        key  liveness  loudness  mode  speechiness    tempo  time_signature  \
0         1    0.0812    -7.678     1       0.4090  203.927               4   
1         8    0.2860    -7.259     1       0.4570  159.009               4   
2         9    0.0824    -5.890     0       0.0454  114.966               4   
3         5    0.0994   -12.118     0       0.0701  123.003               4   
4         7    0.0332   -10.277     1       0.0661  120.047               4   
...     ...       ...       ...   ...          ...      ...             ...   
130984    8    0.0866   -13.522     1       0.0370  110.022               4   
130985    6    0.3620    -2.782     1       0.0718  148.079               4   
130986    2    0.1070    -9.462     1       0.0473  171.217               4   
130987    2    0.8910    -8.466     1       0.0321  125.800               3   
130988    5    0.2720    -5.336     1       0.1490  129.932               4   

        valence  
0        0.1180  
1        0.3710  
2        0.3820  
3        0.6410  
4        0.9280  
...         ...  
130984   0.0375  
130985   0.8200  
130986   0.3650  
130987   0.1050  
130988   0.1930  

[130989 rows x 13 columns]

In [37]:
tree = KDTree(df_stand, leaf_size=10)

In [38]:
df_stand.loc[['6Wosx2euFPMT14UXiWudMy']]

acousticness  danceability  duration_ms    energy  \
6Wosx2euFPMT14UXiWudMy     -0.918477       0.11326    -0.599202  0.590537   

                        instrumentalness       key  liveness  loudness  \
6Wosx2euFPMT14UXiWudMy          -0.62137  1.045559 -0.670599   0.62356   

                            mode  speechiness     tempo  time_signature  \
6Wosx2euFPMT14UXiWudMy -1.245314    -0.535547 -0.149273        0.235377   

                         valence  
6Wosx2euFPMT14UXiWudMy -0.222647

In [39]:
def nearest_by_id(id, k=3):
    return tree.query(df_stand.loc[[id]], k=k)[1]

In [40]:
# Note that it's producing indices. We can lookup their corresponding IDs in df_atts if need be
nearest_by_id('6Wosx2euFPMT14UXiWudMy')

array([[    2, 75037, 44660]])

In [41]:
df_feats.iloc[2]

acousticness             0.0250
danceability             0.6030
duration_ms         138913.0000
energy                   0.7230
instrumentalness         0.0000
key                      9.0000
liveness                 0.0824
loudness                -5.8900
mode                     0.0000
speechiness              0.0454
tempo                  114.9660
time_signature           4.0000
valence                  0.3820
Name: 2, dtype: float64

In [42]:
df_feats.iloc[112174]

acousticness             0.0846
danceability             0.6110
duration_ms         182741.0000
energy                   0.7960
instrumentalness         0.0000
key                     10.0000
liveness                 0.0907
loudness                -3.4340
mode                     0.0000
speechiness              0.0335
tempo                  102.0170
time_signature           4.0000
valence                  0.3960
Name: 112174, dtype: float64

In [43]:
df_feats.iloc[119422]

acousticness             0.0188
danceability             0.6230
duration_ms         175223.0000
energy                   0.6500
instrumentalness         0.0000
key                     11.0000
liveness                 0.0687
loudness                -3.7860
mode                     0.0000
speechiness              0.0320
tempo                  112.0720
time_signature           4.0000
valence                  0.3660
Name: 119422, dtype: float64

In [44]:
df_atts.iloc[6293]

artist_name                WhoMadeWho
track_id       1UglHGkDf43W58XZKdnqbE
track_name                    Dynasty
Name: 6293, dtype: object

In [45]:
# These seem much more similar

In [46]:
from joblib import dump

In [47]:
dump(spoty_scalar, 'SpotyScaled.joblib')

['SpotyScaled.joblib']

In [48]:
# Pickle Tree
dump(tree, 'SpotifyKDTree.joblib')

['SpotifyKDTree.joblib']

In [49]:
# Export standardized features for later use
df_stand.to_csv("standardized_features.csv")

In [50]:
# To use everything from scratch, we can do the following;
# Import standardized data (note the index column)
sdf = pd.read_csv("standardized_features.csv", index_col=0)

# Import picked model
from joblib import load
Spot_KDTree = load('SpotifyKDTree.joblib')

# Make functions using model
def new_nearest_by_id(id, k=3):
    return Spot_KDTree.query(sdf.loc[[id]], k=k)[1]

In [51]:
new_nearest_by_id('6Wosx2euFPMT14UXiWudMy')

array([[    2, 75037, 44660]])

In [52]:
new_nearest_by_id('3J2Jpw61sO7l6Hc7qdYV91')

array([[    3,    36, 50802]])

In [53]:
df

artist_name                track_id  \
0                      YG  2RM4jf1Xa9zPgMGRDiht8O   
1                      YG  1tHDG53xJNGsItRA3vfVgs   
2                   R3HAB  6Wosx2euFPMT14UXiWudMy   
3              Chris Cooq  3J2Jpw61sO7l6Hc7qdYV91   
4              Chris Cooq  2jbYvQCyPgX3CdmAzeVeuS   
...                   ...                     ...   
130984         Instupendo  1xm8J6EFMA6N8JDqH8vzuz   
130985  Mitchell Tenpenny  4NwmHBjPb9i9N3naLCMVCG   
130986      Cold War Kids  5Xo8AsEz0gpW6Rpo2jXvBN   
130987  Elevation Worship  4Fnz6vDqufd3ens0Gf9LC5   
130988          Jae Mansa  2tolmRzbUfgL5KRplIqHlu   

                                            track_name  acousticness  \
0       Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj      0.005820   
1                         BAND DRUM (feat. A$AP Rocky)      0.024400   
2                                        Radio Silence      0.025000   
3                                              Lactose      0.029400   
4                                  Same - Original mix      0.000035   
...                                                ...           ...   
130984                                             Boy      0.820000   
130985                    Truck I Drove in High School      0.070400   
130986                             Goodnight Tennessee      0.064000   
130987                           Hallelujah Here Below      0.029200   
130988                                         I Swear      0.101000   

        danceability  duration_ms  energy  instrumentalness  key  liveness  \
0              0.743       238373   0.339             0.000    1    0.0812   
1              0.846       214800   0.557             0.000    8    0.2860   
2              0.603       138913   0.723             0.000    9    0.0824   
3              0.800       125381   0.579             0.912    5    0.0994   
4              0.783       124016   0.792             0.878    7    0.0332   
...              ...          ...     ...               ...  ...       ...   
130984         0.418       235771   0.204             0.936    8    0.0866   
130985         0.614       209032   0.952             0.000    6    0.3620   
130986         0.343       324920   0.487             0.000    2    0.1070   
130987         0.380       423377   0.399             0.000    2    0.8910   
130988         0.834       240353   0.517             0.000    5    0.2720   

        loudness  mode  speechiness    tempo  time_signature  valence  \
0         -7.678     1       0.4090  203.927               4   0.1180   
1         -7.259     1       0.4570  159.009               4   0.3710   
2         -5.890     0       0.0454  114.966               4   0.3820   
3        -12.118     0       0.0701  123.003               4   0.6410   
4        -10.277     1       0.0661  120.047               4   0.9280   
...          ...   ...          ...      ...             ...      ...   
130984   -13.522     1       0.0370  110.022               4   0.0375   
130985    -2.782     1       0.0718  148.079               4   0.8200   
130986    -9.462     1       0.0473  171.217               4   0.3650   
130987    -8.466     1       0.0321  125.800               3   0.1050   
130988    -5.336     1       0.1490  129.932               4   0.1930   

        popularity  
0               15  
1                0  
2               56  
3                0  
4                0  
...            ...  
130984          51  
130985          47  
130986          45  
130987          49  
130988          53  

[130989 rows x 17 columns]

In [55]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

In [71]:
def get_features(id):
    CLIENT_ID = "2f444856f43c47f6ba712993205ae254"
    CLIENT_SECRET = "9757ea527ba14afc974f8c3ecb09458f"

    credentials = oauth2.SpotifyClientCredentials(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET)

    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)

    feat = spotify.audio_features(tracks=id)
    pop = spotify.track(id)
    i = feat[0]
    feat_dict = {'acousticness':i['acousticness'], 'danceability':i['danceability'],
                'energy':i['energy'], 'instrumentalness':i['instrumentalness'], 'key':i['key'],'liveness':i['liveness'],
                'loudness':i['loudness'], 'mode':i['mode'], 'speechiness':i['speechiness'], 'tempo':i['tempo'],
                'time_signature':i['time_signature'], 'valence':i['valence'], 'popularity':pop['popularity']}
    return(feat_dict)

In [72]:
get_features('26Y1lX7ZOpw9Ql3gGAlqLK')

{'acousticness': 0.00115,
 'danceability': 0.81,
 'energy': 0.417,
 'instrumentalness': 0.919,
 'key': 9,
 'liveness': 0.106,
 'loudness': -10.783,
 'mode': 0,
 'speechiness': 0.0793,
 'tempo': 120.025,
 'time_signature': 4,
 'valence': 0.837,
 'popularity': 0}

In [59]:
pd.DataFrame([[]], )

Empty DataFrame
Columns: []
Index: [0]

In [60]:
df_stand.loc[['6Wosx2euFPMT14UXiWudMy']]

acousticness  danceability  duration_ms    energy  \
6Wosx2euFPMT14UXiWudMy     -0.918477       0.11326    -0.599202  0.590537   

                        instrumentalness       key  liveness  loudness  \
6Wosx2euFPMT14UXiWudMy          -0.62137  1.045559 -0.670599   0.62356   

                            mode  speechiness     tempo  time_signature  \
6Wosx2euFPMT14UXiWudMy -1.245314    -0.535547 -0.149273        0.235377   

                         valence  
6Wosx2euFPMT14UXiWudMy -0.222647

In [61]:
spoty_scalar.transform(pd.DataFrame([get_features('26Y1lX7ZOpw9Ql3gGAlqLK')], index=['26Y1lX7ZOpw9Ql3gGAlqLK']))

array([[ -0.98747862,   1.20259966,  -1.7276266 ,   1.34332005,
         24.35974575,  -1.42306013, -65.42785412,   1.5233613 ,
         -1.08288159, 964.73732218,  -3.82920887,  -5.91193256,
         -1.69713023]])

In [62]:
spoty_scalar.transform([list(get_features('26Y1lX7ZOpw9Ql3gGAlqLK').values())])

array([[ -0.98747862,   1.20259966,  -1.7276266 ,   1.34332005,
         24.35974575,  -1.42306013, -65.42785412,   1.5233613 ,
         -1.08288159, 964.73732218,  -3.82920887,  -5.91193256,
         -1.69713023]])

In [63]:
Spot_KDTree.query(feats, k=8)[1][0,1:]

NameError: name 'feats' is not defined

In [64]:
def get_recs_1(id, k=7):
    feats = list(get_features(id).values())
    scaled_feats = spoty_scalar.transform([feats])
    
    top_k_ind = Spot_KDTree.query(scaled_feats, k=(k+1))[1][0,1:]
    
    top_k_id = list(map(lambda x: lines[x][:-1], top_k_ind))
    
    return top_k_id

In [65]:
def get_recs_2(id, k=7):
    feats = list(get_features(id).values())
    scaled_feats = spoty_scalar.transform([feats])
    
    top_k_ind = Spot_KDTree.query(scaled_feats, k=(k+1))[1][0,1:]
    
    top_k_id = list(map(lambda x: df_atts.loc[[x]]['track_id'].iloc[0], top_k_ind))
    
    return top_k_id

In [66]:
get_recs_1('6Wosx2euFPMT14UXiWudMy')

NameError: name 'lines' is not defined

In [68]:
get_recs_2('4PmMVdIFpJTB9WQbmILf4p')

['5GFtAqMznNdqTq3qe74GwK',
 '1pSrMP1ZtxUvtS9oYNpHP6',
 '5I2eN1a0HI3ZanK2OwdVyt',
 '3nO3oAWMK3lbO3LJadCjao',
 '0fxrG8aI019lrhLXGUqmFt',
 '2nYonPFoA9ts1Re7U7ozDQ',
 '6VW9lyJZKaFSVYDbGbR2S1']

In [103]:
df_atts['track_id'].values

array(['2RM4jf1Xa9zPgMGRDiht8O', '1tHDG53xJNGsItRA3vfVgs',
       '6Wosx2euFPMT14UXiWudMy', ..., '4TWlUuFk81NGUNKwndyS5Q',
       '5iGBXzOoRo4sBTy8wdzMyK', '7LNtyuekYHiZ99UxkrfCQR'], dtype=object)

In [107]:
f = open("song_ids.txt","w+")

In [108]:
for i in df_atts['track_id'].values:
     f.write(str(i)+"\n")

In [126]:
df_atts['track_id'].values

array(['2RM4jf1Xa9zPgMGRDiht8O', '1tHDG53xJNGsItRA3vfVgs',
       '6Wosx2euFPMT14UXiWudMy', ..., '4TWlUuFk81NGUNKwndyS5Q',
       '5iGBXzOoRo4sBTy8wdzMyK', '7LNtyuekYHiZ99UxkrfCQR'], dtype=object)

'2RM4jf1Xa9zPgMGRDiht8O'

In [178]:
i=1002
print(df_atts.loc[[i]]['track_id'].iloc[0])
print(lines[i])

1FinlzvMBkucxM4Ow5zZSs
1FinlzvMBkucxM4Ow5zZSs



In [110]:
f=open('song_ids.txt')
lines=f.readlines()

In [118]:
lines[1][:-1]

'1tHDG53xJNGsItRA3vfVgs'

In [ ]:
numpy.linalg.norm(a-b)

In [75]:
spotify.audio_features('4PmMVdIFpJTB9WQbmILf4p')

NameError: name 'spotify' is not defined

In [76]:
    CLIENT_ID = "2f444856f43c47f6ba712993205ae254"
    CLIENT_SECRET = "9757ea527ba14afc974f8c3ecb09458f"

    credentials = oauth2.SpotifyClientCredentials(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET)

    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)

In [77]:
spotify.audio_features('4PmMVdIFpJTB9WQbmILf4p')

[{'danceability': 0.578,
  'energy': 0.666,
  'key': 7,
  'loudness': -7.1,
  'mode': 1,
  'speechiness': 0.435,
  'acousticness': 0.0496,
  'instrumentalness': 0,
  'liveness': 0.0335,
  'valence': 0.591,
  'tempo': 195.13,
  'type': 'audio_features',
  'id': '4PmMVdIFpJTB9WQbmILf4p',
  'uri': 'spotify:track:4PmMVdIFpJTB9WQbmILf4p',
  'track_href': 'https://api.spotify.com/v1/tracks/4PmMVdIFpJTB9WQbmILf4p',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4PmMVdIFpJTB9WQbmILf4p',
  'duration_ms': 235533,
  'time_signature': 4}]

In [85]:
spotify.track('1pSrMP1ZtxUvtS9oYNpHP6')

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3AQJBoKtkFiM2nQfCpY3jp'},
    'href': 'https://api.spotify.com/v1/artists/3AQJBoKtkFiM2nQfCpY3jp',
    'id': '3AQJBoKtkFiM2nQfCpY3jp',
    'name': 'Brother Larmatine',
    'type': 'artist',
    'uri': 'spotify:artist:3AQJBoKtkFiM2nQfCpY3jp'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/49yM77WYUiJcFMttP56j5v'},
  'href': 'https://api.spotify.com/v1/albums/49yM77WYUiJcFMttP56j5v',
  'id': '49yM77WYUiJcFMttP56j5v',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b27317396feaa095919a93059f5b',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e0217396feaa095919a93059f5b',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d0000485117396feaa095919a93059f5b',
    'width': 64}],
  'name': 'Chapterxivi',
  'release_date': '2018-01-07',
  'relea